In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import MultipleLocator
# plt.style.use('seaborn-paper')
import seaborn as sns
import joblib
# plt.rcParams['font.family']='Times New Roman,Microsoft YaHei'# 设置字体族，中文为微软雅黑，英文为Times New Roman
plt.rcParams['font.sans-serif'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'stix'  # 设置数学公式字体为stix
plt.rcParams["text.usetex"] = False
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
from sklearn.metrics import mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.patheffects as path_effects

In [2]:
datajh=pd.read_csv("groupedjhS.csv")
datacm=pd.read_csv("groupedcmS.csv")
datajhsoa=pd.read_csv("groupedjhSOA.csv")
datacmsoa=pd.read_csv("groupedcmSOA.csv")

In [3]:
#温度影响
VOCs = ['Methyl Mercaptan', '1,3-Butadiene', 'Butene', 'Acetone/Butane', 'n-Propanol',
        'Dimethyl Sulfide/Ethyl Mercaptan', 'Chloroethane', 'Isoprene', 'Pentene', 'Pentane/Isopentane',
        'Dimethylformamide', 'Ethyl Formate', 'Carbon Disulfide/Propyl Mercaptan', 'Benzene', 'Cyclohexene',
        'Hexene/Methylcyclopentane', 'n-Hexane/Dimethylbutane', 'Ethyl Sulfide/Butyl Mercaptan', 'Toluene', 'Aniline',
        'Dimethyl Disulfide', '1,1-Dichloroethylene', 'Methylcyclohexane', 'n-Heptane', 'Triethylamine',
        'n-Propyl Acetate', 'Diethylene Triamine', 'Styrene', 'Xylene/Ethylbenzene', '1,3-Dichloropropene', 'n-Octane',
        'n-Butyl Acetate', 'Hexyl Mercaptan', 'Xylenol', 'Trichloroethylene', 'Diethylbenzene', 'Methyl Benzoate',
        'Trimethyl Phosphate', 'n-Decanol', 'Dichlorobenzene', 'Diethyl Aniline', 'Undecane', 'Tetrachloroethylene',
        'n-Dodecane', 'Dibromomethane', '1,2,4-Trichlorobenzene', 'n-Tridecane', '1,2-Dibromoethane']

datajhsoa["SOA"] = 0
datajh["SOA"] = 0
for i in datajhsoa.columns[(datajhsoa.columns.get_loc("0.25um")):(datajhsoa.columns.get_loc("0.30um"))]:
    datajhsoa["SOA"] = datajhsoa["SOA"] + datajhsoa[i]
    datajh["SOA"] = datajh["SOA"] + datajh[i]
datajhsoa["CSOA"] = 0
for i in datajhsoa.columns[(datajhsoa.columns.get_loc("C0.25um")):(datajhsoa.columns.get_loc("C0.30um"))]:
    datajhsoa["CSOA"] = datajhsoa["CSOA"] + datajhsoa[i]
datacmsoa["SOA"] = 0
datacm["SOA"] = 0
for i in datacmsoa.columns[(datacmsoa.columns.get_loc("0.25um")):(datacmsoa.columns.get_loc("0.30um"))]:
    datacmsoa["SOA"] = datacmsoa["SOA"] + datacmsoa[i]
    datacm["SOA"] = datacm["SOA"] + datacm[i]
datacmsoa["CSOA"] = 0
for i in datacmsoa.columns[(datacmsoa.columns.get_loc("C0.25um")):(datacmsoa.columns.get_loc("C0.30um"))]:
    datacmsoa["CSOA"] = datacmsoa["CSOA"] + datacmsoa[i]
datajh['place'] = 'JH'
datacm['place'] = 'CM'
dataall = pd.concat([datajh, datacm], axis=0)
dataall.columns = ['Time', 'TVOCs', 'Methyl Mercaptan', '1,3-Butadiene', 'Butene', 'Acetone/Butane', 'n-Propanol',
                   'Dimethyl Sulfide/Ethyl Mercaptan', 'Chloroethane', 'Isoprene', 'Pentene', 'Pentane/Isopentane',
                   'Dimethylformamide', 'Ethyl Formate', 'Carbon Disulfide/Propyl Mercaptan', 'Benzene', 'Cyclohexene',
                   'Hexene/Methylcyclopentane', 'n-Hexane/Dimethylbutane', 'Ethyl Sulfide/Butyl Mercaptan', 'Toluene',
                   'Aniline', 'Dimethyl Disulfide', '1,1-Dichloroethylene', 'Methylcyclohexane', 'n-Heptane',
                   'Triethylamine', 'n-Propyl Acetate', 'Diethylene Triamine', 'Styrene', 'Xylene/Ethylbenzene',
                   '1,3-Dichloropropene', 'n-Octane', 'n-Butyl Acetate', 'Hexyl Mercaptan', 'Xylenol',
                   'Trichloroethylene', 'Diethylbenzene', 'Methyl Benzoate', 'Trimethyl Phosphate', 'n-Decanol',
                   'Dichlorobenzene', 'Diethyl Aniline', 'Undecane', 'Tetrachloroethylene', 'n-Dodecane',
                   'Dibromomethane', '1,2,4-Trichlorobenzene', 'n-Tridecane', '1,2-Dibromoethane', '0.25um', '0.28um',
                   '0.30um', '0.35um', '0.40um', '0.45um', '0.50um', '0.58um', '0.65um', '0.70um', '0.80um', '1.00um',
                   '1.30um', '1.60um', '2.00um', '2.50um', '3.00um', '3.50um', '4.00um', '5.00um', '6.50um', '7.50um',
                   '8.50um', '10.00um', '12.50um', '15.00um', '17.50um', '20.00um', '25.00um', '30.00um', '32.00um',
                   'PM10', 'PM2.5', 'PM1', 'SO2', 'NOx', 'NO', 'NO2', 'CO', 'O3', 'NO2.1', 'NegativeOxygenIons',
                   'Radiation', 'Temperature', 'Humidity', 'WindSpeed', 'Hour_Min_Sec', 'Hour_Min', 'Hour', 'Month',
                   'Day', 'Datetime', 'seconds', 'SOA', 'place']


In [4]:
from scipy.optimize import curve_fit
from scipy.stats import zscore
from matplotlib.ticker import FuncFormatter, MaxNLocator
import matplotlib.dates as mdates
from scipy.stats import t


# 定义与均值的关系函数（二次函数）
def mean_relation(T, Q0, a, v0):
    return Q0 + (a * T**2) / 2 + T * v0

# 定义与方差的关系函数（三次函数）
def std_dev_relation(T, k, sigma0):
    return (k**2 * T**3) / 3 + k * T**2 * sigma0 + T * sigma0**2


# 定义去除离群值的函数（使用 Z-score 方法）
def remove_outliers(data):
    z_scores = zscore(data)
    return data[(np.abs(z_scores) < 3)]  # 通常使用 3 作为 Z-score 的阈值
    
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon
from shapely.ops import unary_union
from scipy.stats import norm

def compute_area(points):
    # 计算点的凸包区域
    hull = ConvexHull(points)
    polygon = Polygon(points[hull.vertices])
    return polygon.area

def monte_carlo_r_squared_area(T, mean_params, std_dev_params, real_data, num_simulations=1000):
    original_points = np.column_stack((T, real_data))
    original_area = compute_area(original_points)

    simulation_areas = []
    overlap_areas = []

    for _ in range(num_simulations):
        simulated_path = norm.rvs(
            loc=mean_relation(T, *mean_params),
            scale=np.sqrt(std_dev_relation(T, *std_dev_params)),
            size=len(T)
        )
        simulated_points = np.column_stack((T, simulated_path))
        sim_area = compute_area(simulated_points)
        simulation_areas.append(sim_area)

        original_polygon = Polygon(original_points[ConvexHull(original_points).vertices])
        simulated_polygon = Polygon(simulated_points[ConvexHull(simulated_points).vertices])
        intersection_area = original_polygon.intersection(simulated_polygon).area
        overlap_areas.append(intersection_area)
    
    mean_overlap_area = np.mean(overlap_areas)
    mean_simulation_area = np.mean(simulation_areas)

    # Calculate R² based on area overlap
    r_squared_area = mean_overlap_area / original_area
    return min(max(r_squared_area, 0), 1)  # Ensure R² is in [0, 1]

# 定义异常值清理函数
def clean_data(df, columns, threshold=3):
    for col in columns:
        df = df[np.abs(zscore(df[col])) < threshold]
    return df.reset_index(drop=True)

def hour_min_to_float(hour_min_str):
    hour, minute = map(int, hour_min_str.split(":"))
    return hour + minute / 60.0

import re
def scientific_notation_with_superscript(value, precision=3):
    # 如果值为零，直接返回零字符串格式
    if value == 0:
        return f"{value:.{precision}f}"
    
    # 格式化为科学记数法，并去掉指数部分的前导零
    formatted_value = f"{value:.{precision}e}"
    formatted_value = formatted_value.replace('e+', 'x10^').replace('e-', 'x10^-')
    formatted_value = re.sub(r'x10\^([-+]?)0*(\d+)', r'x10^\1\2', formatted_value)  # 保留负号
    
    # 将指数部分的数字替换为 Unicode 上标
    superscript_map = str.maketrans("0123456789-", "⁰¹²³⁴⁵⁶⁷⁸⁹⁻")
    match = re.search(r'x10\^([-+]?\d+)', formatted_value)
    if match:
        exponent = match.group(1).translate(superscript_map)
        formatted_value = re.sub(r'x10\^[-+]?\d+', f'x10{exponent}', formatted_value)
         # 如果是 x10⁰ 则移除
        if 'x10⁰' in formatted_value:
            formatted_value = formatted_value.replace('x10⁰', '')
    else:
        # 如果没有 x10^，直接返回数值部分
        formatted_value = formatted_value.split('x10')[0]
         # 如果是 x10⁰ 则移除
        if 'x10⁰' in formatted_value:
            formatted_value = formatted_value.replace('x10⁰', '')
        
    return formatted_value

from scipy.stats import norm, expon, gamma, lognorm, beta, kstest, shapiro
def normal_distribution_fit_and_test(grouped_by_hour):
    normality_results = {}
    for hour, group in grouped_by_hour:
        # Fit normal distribution and get parameters
        params = norm.fit(group)
        # Perform Shapiro-Wilk test
        _, p_value = shapiro(group)
        # Check if data is normally distributed based on p-value
        is_normal = True if p_value > 0.05 else False
        normality_results[hour] = (
        params[0], params[1], p_value, is_normal)  # params[0]: mean, params[1]: standard deviation

    normality_df = pd.DataFrame(normality_results).T
    normality_df.columns = ["Mean", "Standard Deviation", "P-Value", "Is Normal"]
    normality_df.reset_index(inplace=True)
    normality_df.rename(columns={'index': 'Hour'}, inplace=True)

    return normality_df

In [5]:
# 从 dataall 中筛选出 JH 和 CM 地点的数据
data_jh = dataall[dataall['place'] == 'JH'].copy()
data_cm = dataall[dataall['place'] == 'CM'].copy()
data=dataall.groupby(["Hour_Min"]).mean(numeric_only=True).reset_index()

In [6]:

def create_models_for_location(data_location):
    # 计算每小时的均值
    datare = data_location.groupby(["Hour_Min"]).mean(numeric_only=True).reset_index()
    dataj = pd.DataFrame({
        "Time": pd.to_datetime(datare["Hour_Min"]),
        "T": datare["Temperature"],
        "hv": datare["Radiation"],
        "RH": datare["Humidity"],
        "O3": datare["O3"],
        "NOx": datare["NOx"],
        "SO2": datare["SO2"],
        "SOA": datare["SOA"],
        "K": 1,
        'Isoprene': datare["Isoprene"]
    })

    # 计算交互变量
    dataj["HNO3"] = dataj["RH"] * dataj["NOx"]
    dataj["H2SO4"] = dataj["RH"] * dataj["SO2"]
    dataj["H2SO403"] = dataj["RH"] * dataj["SO2"] * dataj["O3"]
    dataj["HNO3O3"] = dataj["RH"] * dataj["NOx"] * dataj["O3"]
    dataj["O3hv"] = dataj["O3"] * dataj["hv"]

    variables_to_regress = ["HNO3", "H2SO4", "H2SO403", "HNO3O3", "O3hv", "K", "hv"]

    # 模型 1：直接使用观测到的 Isoprene 浓度
    dataj["BVOCs"] = datare["Isoprene"]
    for var in variables_to_regress:
        dataj[var + "_BVOCs"] = dataj[var] * dataj["Isoprene"]
    

    # 模型 2：使用温度拟合 Isoprene 的均值和方差，再以方差为权重拟合 SOA
    data_grouped = data_location.groupby(["Hour_Min"]).mean(numeric_only=True).reset_index()
    data_grouped["Concentration"] = data_grouped["Isoprene"]
    grouped_by_hour = data_grouped.groupby('Hour')
    normality_df = normal_distribution_fit_and_test(grouped_by_hour["Isoprene"])
    normality_df['T'] = data_grouped.groupby('Hour').mean(numeric_only=True)["Temperature"]

    # 提取数据
    T = normality_df['T'].values
    mean_values = normality_df['Mean'].astype("float").values
    std_dev_values = normality_df['Standard Deviation'].astype("float").values ** 2

    # 去除离群值
    mean_values_filtered = remove_outliers(mean_values)
    std_dev_values_filtered = remove_outliers(std_dev_values)
    T_filtered_mean = T[np.isin(mean_values, mean_values_filtered)]
    T_filtered_std_dev = T[np.isin(std_dev_values, std_dev_values_filtered)]

    # 拟合模型
    params_mean, _ = curve_fit(mean_relation, T_filtered_mean, mean_values_filtered, method='trf', maxfev=10000)
    params_std_dev, _ = curve_fit(std_dev_relation, T_filtered_std_dev, std_dev_values_filtered, method='trf')

    # 计算 fitted Isoprene 和权重
    mean_isoprene = mean_relation(dataj["T"], *params_mean)
    std_dev_isoprene = std_dev_relation(dataj["T"], *params_std_dev)
    dataj["Isoprene_fitted"] = mean_isoprene
    weights = 1 / std_dev_isoprene  # 使用方差的倒数作为权重

    for var in variables_to_regress:
        dataj[var + "_BVOCs"] = dataj[var] * dataj["Isoprene_fitted"]

    X2 = dataj[[var + "_BVOCs" for var in variables_to_regress]]
    Y2 = dataj["SOA"]
    model2 = sm.WLS(Y2, X2, weights=weights).fit(cov_type='HC3')


    # 输出模型
    return model2,params_mean,params_std_dev


In [7]:
dataall['SOA'].mean()/dataall['PM2.5'].mean()

0.12696495955123446

In [8]:
# 构建 JH 和 CM 地点的模型
modeljh,params_meanjh,params_std_devjh= create_models_for_location(data_jh)
modelcm,params_meancm,params_std_devcm=create_models_for_location(data_cm)

C:\Users\aa271\AppData\Local\Temp\ipykernel_26556\4240856103.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  "Time": pd.to_datetime(datare["Hour_Min"]),
C:\Users\aa271\AppData\Local\Temp\ipykernel_26556\4240856103.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  "Time": pd.to_datetime(datare["Hour_Min"]),


In [9]:
modelall,params_mean,params_std_dev=create_models_for_location(dataall)

C:\Users\aa271\AppData\Local\Temp\ipykernel_26556\4240856103.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  "Time": pd.to_datetime(datare["Hour_Min"]),


In [10]:
# data_location=data_jh.copy()
# params_mean=params_meanjh
# params_std_dev=params_std_devjh
# model=modeljh

# data_location=data_cm.copy()
# params_mean=params_meancm
# params_std_dev=params_std_devcm
# model=modelcm

data_location=dataall.copy()
params_mean=params_mean
params_std_dev=params_std_dev
model=modelall

# 计算每小时的均值
datare = data_location.groupby(["Hour_Min"]).mean(numeric_only=True).reset_index()
dataj = pd.DataFrame({
    "Time": datare["Hour_Min"],
    "T": datare["Temperature"],
    "hv": datare["Radiation"],
    "RH": datare["Humidity"],
    "O3": datare["O3"],
    "NOx": datare["NOx"],
    "SO2": datare["SO2"],
    "SOA": datare["SOA"],
    "K": 1,
    'Isoprene': datare["Isoprene"]
})

# 计算交互变量
dataj["HNO3"] = dataj["RH"] * dataj["NOx"]
dataj["H2SO4"] = dataj["RH"] * dataj["SO2"]
dataj["H2SO403"] = dataj["RH"] * dataj["SO2"] * dataj["O3"]
dataj["HNO3O3"] = dataj["RH"] * dataj["NOx"] * dataj["O3"]
dataj["O3hv"] = dataj["O3"] * dataj["hv"]

variables_to_regress = ["HNO3", "H2SO4", "H2SO403", "HNO3O3", "O3hv", "K", "hv"]
        

In [11]:
dataj["Isoprene_fitted"] = mean_relation(dataj["T"], *params_mean)
for var in variables_to_regress:
    dataj[var + "_BVOCs"] = dataj[var] * dataj["Isoprene_fitted"]

X = dataj[[var + "_BVOCs" for var in variables_to_regress]]
Y = model.predict(X)


In [12]:
# 生成50条随机的温度序列
num_simulations = 1000
dataset=[]

# 生成原始情况的SOA
mean_isoprene = mean_relation(dataj["T"], *params_mean)
std_dev_isoprene = std_dev_relation(dataj["T"], *params_std_dev)

for i in range(num_simulations):
    random_BVOCs = np.random.normal(loc=mean_isoprene, scale=std_dev_isoprene, size=dataj.shape[0])
    dataj["Isoprene_fitted"] = random_BVOCs
    for var in variables_to_regress:
        dataj[var + "_BVOCs"] = dataj[var] * dataj["Isoprene_fitted"]
    
    X = dataj[[var + "_BVOCs" for var in variables_to_regress]]
    Y = model.predict(X)
    
    temp_data = pd.DataFrame({
        "Time": dataj["Time"],
        "Temperature": dataj["T"],
        "BVOCs": dataj["Isoprene_fitted"],
        "SOA": Y
    })
    dataset.append(temp_data)

# 生成1.5°C温升情景的SOA
mean_isoprene = mean_relation(dataj["T"] + 1.5, *params_mean)
std_dev_isoprene = std_dev_relation(dataj["T"] + 1.5, *params_std_dev)

for i in range(num_simulations):
    random_BVOCs = np.random.normal(loc=mean_isoprene, scale=std_dev_isoprene, size=dataj.shape[0])
    dataj["Isoprene_fitted"] = random_BVOCs
    for var in variables_to_regress:
        dataj[var + "_BVOCs"] = dataj[var] * dataj["Isoprene_fitted"]
    
    X = dataj[[var + "_BVOCs" for var in variables_to_regress]]
    Y = model.predict(X)
    
    temp_data = pd.DataFrame({
        "Time": dataj["Time"],
        "Temperature": dataj["T"] + 1.5,
        "BVOCs": dataj["Isoprene_fitted"],
        "SOA": Y
    })
    dataset.append(temp_data)

# 生成2°C温升情景的SOA
mean_isoprene = mean_relation(dataj["T"] + 2, *params_mean)
std_dev_isoprene = std_dev_relation(dataj["T"] + 2, *params_std_dev)

for i in range(num_simulations):
    random_BVOCs = np.random.normal(loc=mean_isoprene, scale=std_dev_isoprene, size=dataj.shape[0])
    dataj["Isoprene_fitted"] = random_BVOCs
    for var in variables_to_regress:
        dataj[var + "_BVOCs"] = dataj[var] * dataj["Isoprene_fitted"]
    
    X = dataj[[var + "_BVOCs" for var in variables_to_regress]]
    Y = model.predict(X)
    
    temp_data = pd.DataFrame({
        "Time": dataj["Time"],
        "Temperature": dataj["T"] + 2,
        "BVOCs": dataj["Isoprene_fitted"],
        "SOA": Y
    })
    dataset.append(temp_data)

# 生成3°C温升情景的SOA
mean_isoprene = mean_relation(dataj["T"] + 3, *params_mean)
std_dev_isoprene = std_dev_relation(dataj["T"] + 3, *params_std_dev)

for i in range(num_simulations):
    random_BVOCs = np.random.normal(loc=mean_isoprene, scale=std_dev_isoprene, size=dataj.shape[0])
    dataj["Isoprene_fitted"] = random_BVOCs
    for var in variables_to_regress:
        dataj[var + "_BVOCs"] = dataj[var] * dataj["Isoprene_fitted"]
    
    X = dataj[[var + "_BVOCs" for var in variables_to_regress]]
    Y = model.predict(X)
    
    temp_data = pd.DataFrame({
        "Time": dataj["Time"],
        "Temperature": dataj["T"] + 3,
        "BVOCs": dataj["Isoprene_fitted"],
        "SOA": Y
    })
    dataset.append(temp_data)

In [13]:
# # 用于存储每个 model 的均值超标和总体超标比例
# exceedance_dict = {}
# 
# # 总时间点数量
# total_time_points = dataset[0].shape[0]
# 
# # 遍历每个模拟情景数据集并计算超标比例
# for data in dataset:
#     # 根据温度值判断当前数据的模式
#     if data["Temperature"].iloc[0] == dataj["T"].iloc[0]:
#         model_label = "Baseline"
#     elif data["Temperature"].iloc[0] == dataj["T"].iloc[0] + 1.5:
#         model_label = "+1.5°C Warming"
#     elif data["Temperature"].iloc[0] == dataj["T"].iloc[0] + 2:
#         model_label = "+2.0°C Warming"
#     elif data["Temperature"].iloc[0] == dataj["T"].iloc[0] + 3:
#         model_label = "+3.0°C Warming"
#     else:
#         continue  # 跳过不匹配任何情景的数据
# 
#     # 初始化 model 的记录
#     if model_label not in exceedance_dict:
#         exceedance_dict[model_label] = {"mean_exceedance": 0, "total_exceedance": 0, "count": 0, "exceed_times": set()}
# 
#     # 计算 mean_exceedance：按 Time 分组计算 SOA 的均值并检查是否超过 5
#     mean_exceedance = (data.groupby("Time")["SOA"].mean() > 5).mean() * 100
#     
#     # 找出该路径中所有超标的时间点
#     exceed_times = set(data.loc[data["SOA"] > 5, "Time"])
# 
#     # 更新超标时间点集合（集合自动去重）
#     exceedance_dict[model_label]["exceed_times"].update(exceed_times)
# 
#     # 累加 mean_exceedance
#     exceedance_dict[model_label]["mean_exceedance"] += mean_exceedance
#     exceedance_dict[model_label]["count"] += 1
# 
# # 计算每个 model 的平均超标比例和总体超标比例
# for model in exceedance_dict:
#     # 计算均值超标比例
#     exceedance_dict[model]["mean_exceedance"] /= exceedance_dict[model]["count"]
#     # 计算总超标比例
#     exceedance_dict[model]["total_exceedance"] = len(exceedance_dict[model]["exceed_times"]) / total_time_points * 100




In [14]:
combined_data = pd.DataFrame()

# 初始化字典，用于跟踪每个 model 的采样次数
sample_counts = {"Baseline": 0, "+1.5°C Warming": 0, "+2.0°C Warming": 0, "+3.0°C Warming": 0}

# 遍历每个模拟情景并添加到 combined_data 中，控制每个 model 最多 10 次
for data in dataset:
    # 根据温度值判断当前数据的模式并添加到新列 'model'
    if data["Temperature"].iloc[0] == dataj["T"].iloc[0]:
        model_label = "Baseline"
    elif data["Temperature"].iloc[0] == dataj["T"].iloc[0] + 1.5:
        model_label = "+1.5°C Warming"
    elif data["Temperature"].iloc[0] == dataj["T"].iloc[0] + 2:
        model_label = "+2.0°C Warming"
    elif data["Temperature"].iloc[0] == dataj["T"].iloc[0] + 3:
        model_label = "+3.0°C Warming"
    else:
        continue  # 如果不匹配任何情景，跳过当前数据

    # 如果当前 model 已经有 10 次采样，则跳过
    if sample_counts[model_label] >= 10:
        continue

    # 设置 model 列，并添加到 combined_data 中
    data["model"] = model_label
    combined_data = pd.concat([combined_data, data], ignore_index=True)
    
    # 更新采样次数
    sample_counts[model_label] += 1

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import patches, lines
from adjustText import adjust_text
import matplotlib.patheffects as path_effects
from statsmodels.nonparametric.smoothers_lowess import lowess

colors = ['#464AA6', '#F2CB05', '#F28A2E', '#BF2633']
# 设置图形尺寸
fig = plt.figure(figsize=(8, 8))
gs = fig.add_gridspec(2,2)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])
ax3 = fig.add_subplot(gs[1, :])




# 第一张图：SOA密度图 (subplot 1)
sns.kdeplot(data=combined_data, x='SOA', hue="model", fill=True, palette=colors,ax=ax1)
ax1.set_title("")
ax1.set_xlabel(r"SOA ($\boldsymbol{μg/m^3}$)", fontsize=13, weight='bold')
ax1.set_ylabel("Density", fontsize=13, weight='bold')
ax1.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
# 优化刻度标签的样式
ax1.tick_params(axis='both', which='major', labelsize=12)
ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.2f}'))  # Y轴格式化显示两位小数
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.2f}'))  # Y轴格式化显示两位小数
ax1.legend_.remove()  



# 第二张图：SOA路径线条图 (subplot 2)
model_label_tex = (
    r"$\boldsymbol{C(T)} [(\boldsymbol{k_{O3}}\boldsymbol{hv} +$" + "\n" +
    r"$\boldsymbol{k_{NOx,O3}}[\boldsymbol{NO3}][\boldsymbol{RH}] +$" + "\n" +
    r"$\boldsymbol{k_{SO2,O3}}[\boldsymbol{SO2}][\boldsymbol{RH}]$)$[\boldsymbol{O3}]$ +" + "\n" +
    r"$\boldsymbol{k_{NOx}}[\boldsymbol{NO3}] [\boldsymbol{RH}] + $" + "\n" +
    r"$\boldsymbol{k_{SO2}}[\boldsymbol{SO2}][\boldsymbol{RH}] +$" + "\n" +
    r"$\boldsymbol{k_c} + \boldsymbol{k_{hv}}\boldsymbol{hv})$"
)

# 遍历每种情景，绘制LOESS平滑曲线和透明散点
for i,model in enumerate(combined_data["model"].unique()):
    model_data = combined_data[combined_data["model"] == model]
    # 使用 LOESS 进行平滑
    loess_result = lowess(model_data["SOA"], model_data["Temperature"], frac=0.3)
    # 绘制平滑曲线
    ax2.plot(
        loess_result[:, 0], loess_result[:, 1],
        label=f"{model} LOESS fit",
        linewidth=2.5,zorder=10,color=colors[i])


# 用于记录每个 model 已绘制的路径数量
plot_count = {"Baseline": 0, "+1.5°C Warming": 0, "+2.0°C Warming": 0, "+3.0°C Warming": 0}
# 遍历每个路径数据集并绘制，确保每个 model 仅绘制 10 条路径
for data in dataset:
    # 确定当前路径的 model 类型
    if data["Temperature"].iloc[0] == dataj["T"].iloc[0]:
        model_label = "Baseline"
    elif data["Temperature"].iloc[0] == dataj["T"].iloc[0] + 1.5:
        model_label = "+1.5°C Warming"
    elif data["Temperature"].iloc[0] == dataj["T"].iloc[0] + 2:
        model_label = "+2.0°C Warming"
    elif data["Temperature"].iloc[0] == dataj["T"].iloc[0] + 3:
        model_label = "+3.0°C Warming"
    else:
        continue  # 跳过不匹配任何情景的数据

    # 如果当前 model 已绘制了 10 条路径，则跳过
    if plot_count[model_label] >= 5:
        continue
    # 绘制路径
    sns.lineplot(data=data, x="Temperature", y="SOA", ax=ax2, color="grey", alpha=0.1, linewidth=0.7, linestyle='-')
    # 更新绘制路径计数
    plot_count[model_label] += 1
    

# 设置标题和坐标轴标签的格式
ax2.set_title("", fontsize=5, weight='bold', pad=15)
ax2.set_xlabel("Temperature (°C)", fontsize=13, weight='bold')
ax2.set_ylabel(r"Average SOA ($\boldsymbol{μg/m^3}$)", fontsize=13, weight='bold')
# 设置网格样式
ax2.grid(True, linestyle='--', linewidth=0.7, alpha=0.6)
# 优化刻度标签的样式
ax2.tick_params(axis='both', which='major', labelsize=12)
ax2.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.2f}'))  # Y轴格式化显示两位小数
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.2f}'))  # Y轴格式化显示两位小数
# 计算 SOA 的均值和标准差
mean_soa = dataall["SOA"].mean()
std_soa = dataall["SOA"].std()
# 绘制均值线
ax2.axhline(y=mean_soa, color='#63E398', linestyle='--', linewidth=2, label="Mean SOA")
# 绘制正负 2 个标准差
ax2.axhline(y=mean_soa + 1 * std_soa, color='#B1CE46', linestyle='--', linewidth=2, label="Mean + 1 Std Dev")
ax2.axhline(y=mean_soa + 2 * std_soa, color='#F1D77E', linestyle='--', linewidth=2, label="Mean + 2 Std Dev")
ax2.text(0.14, 0.93, f'Model\nR² = {modelall.rsquared:.3f}\n'+model_label_tex, transform=ax2.transAxes, fontsize=11, color='black', fontweight='bold', verticalalignment='top')
ax2.legend_.remove()  



#第三张图
# 用于记录每个 model 已绘制的路径数量
plot_count = {"Baseline": 0, "+1.5°C Warming": 0, "+2.0°C Warming": 0, "+3.0°C Warming": 0}
# 遍历每个路径数据集并绘制，确保每个 model 仅绘制 10 条路径
for data in dataset:
    # 确定当前路径的 model 类型
    if data["Temperature"].iloc[0] == dataj["T"].iloc[0]:
        model_label = "Baseline"
    elif data["Temperature"].iloc[0] == dataj["T"].iloc[0] + 1.5:
        model_label = "+1.5°C Warming"
    elif data["Temperature"].iloc[0] == dataj["T"].iloc[0] + 2:
        model_label = "+2.0°C Warming"
    elif data["Temperature"].iloc[0] == dataj["T"].iloc[0] + 3:
        model_label = "+3.0°C Warming"
    else:
        continue  # 跳过不匹配任何情景的数据

    # 如果当前 model 已绘制了 10 条路径，则跳过
    if plot_count[model_label] >= 10:
        continue

    # 绘制路径
    sns.lineplot(data=data, x="Time", y="SOA", ax=ax3, color="grey", alpha=0.3, linewidth=0.7, linestyle='-')

    # 更新绘制路径计数
    plot_count[model_label] += 1
# # 绘制每个情景的均值曲线
sns.lineplot(data=combined_data, x="Time", y="SOA", hue="model", ax=ax3, linewidth=2, palette=colors, marker=None)
# 用于存储注释的文本对象列表
texts = []
# 计算并标注每个情景的均值超标和总体超标比例
for i,model in enumerate(combined_data["model"].unique()):
    model_data = combined_data[combined_data["model"] == model]
    # 获取每个情景的中心位置
    center_time = model_data["Time"].iloc[dataset[0].shape[0] // 2]
    center_soa = model_data["SOA"].mean()
    mean=model_data["SOA"].mean()
    std=model_data["SOA"].std()
    text = ax3.text(
    center_time, center_soa,
    f"{model}\nMean={mean:.2f}\nStd={std:.2f}",
    ha='center', fontsize=12,color='black'
)
    texts.append(text)
    # 添加白色描边效果
    for text in texts:
        text.set_path_effects([
            path_effects.Stroke(linewidth=1.5, foreground="white"),
            path_effects.Normal()
        ])
# 使用 adjust_text 避免文字重叠
adjust_text(
    texts,
    ax=ax3,
    arrowprops=dict(arrowstyle="-|>", color='black', zorder=10),  # 设置箭头的 zorder
)
# 计算 SOA 的均值和标准差
mean_soa = dataall["SOA"].mean()
std_soa = dataall["SOA"].std()
# 绘制均值线
ax3.axhline(y=mean_soa, color='#63E398', linestyle='--', linewidth=2, label="Mean SOA")
# 绘制正负 2 个标准差
ax3.axhline(y=mean_soa + 1 * std_soa, color='#B1CE46', linestyle='--', linewidth=2, label="Mean + 1 Std Dev")
ax3.axhline(y=mean_soa + 2 * std_soa, color='#F1D77E', linestyle='--', linewidth=2, label="Mean + 2 Std Dev")
# 设置标题和标签
ax3.set_title("", fontsize=14, weight='bold')
ax3.set_xlabel("", fontsize=13, weight='bold')
ax3.set_ylabel(r"SOA ($\boldsymbol{μg/m^3}$)", fontsize=13, weight='bold')
ax3.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
# 设置 x 轴的显示间隔为每 4 小时
ax3.set_xticks(ax3.get_xticks()[::90])  
# 优化标签
ax3.tick_params(axis='both', which='major', labelsize=12)
ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.2f}'))  # Y轴格式化显示两位小数
ax3.tick_params(axis='x', labelrotation=45, labelsize=10)
ax3.legend_.remove()  

for ax in [ax1,ax2,ax3]:
    for label in ax.get_xticklabels() + ax.get_yticklabels():
            label.set_fontweight('bold')


# 自定义图例
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
# 自定义图例项
legend_elements = [
    mpatches.Patch(color='#464AA6', label="Baseline"),
    mpatches.Patch(color='#F2CB05', label="+1.5°C Warming"),
    mpatches.Patch(color='#F28A2E', label="+2.0°C Warming"),
    mpatches.Patch(color='#BF2633', label="+3.0°C Warming"),
    Line2D([0], [0], color='#63E398', linestyle='--', linewidth=2, label="Mean SOA"),
    Line2D([0], [0], color='#B1CE46', linestyle='--', linewidth=2, label="Mean + 1 Std Dev"),
    Line2D([0], [0], color='#F1D77E', linestyle='--', linewidth=2, label="Mean + 2 Std Dev"),
    Line2D([0], [0], color='grey', linestyle='-', linewidth=2, alpha=1, label="Monte Carlo\nSimulation")
]
# 在大图右侧添加图例
fig.legend(handles=legend_elements, loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10, prop={'size': 13, 'weight': 'bold'}, frameon=False)


# 调整布局，确保美观
fig.tight_layout()
plt.savefig('随机过程模拟分析.svg',  bbox_inches='tight')
plt.savefig('随机过程模拟分析.jpg',dpi=700,bbox_inches='tight')
plt.show()
